# ●号数を求める

<b>
    [ルール]
    １月１日を含む週を１号とし、以降各週に通し番号が振られる。<br>
</b>

***
***

## ○号数から日にちを算出する関数

In [1]:
# 年（year）,号(gou),曜日(yobi)を指定すれば、日にちを返す関数。
import datetime
def detect_day(year, gou, yobi):
    date_format = '%Y%m%d' # 出力データの形式
    year0101 = str(year)+'0101' # その年の1月1日
    d = datetime.datetime.strptime(year0101, date_format) # datatime型に変える。
    d += datetime.timedelta(days = (gou-1) * 7) # この日を含む週が、求めたい号の週
    # 曜日の調整
    d -= datetime.timedelta(days = datetime.date(int(year), 1, 1).weekday())
    d += datetime.timedelta(days = dict(月=0,火=1,水=2,木=3,金=4,土=5,日=6)[yobi])
    date_string_output = d.strftime(date_format)
    return date_string_output

In [2]:
# 2017年の第1号の木曜日って何月何日か？
detect_day('2017', 1, '木')

'20161229'

***

In [3]:
# 呼び方の対応関係。
tv_dict = dict(NHK='JOAK',
               ETV='JOAB',
               NTV='JOAX',
               TBS='JORX',
               CXT='JOCX',
               EX='JOEX',
               TX='JOTX',
               OUJ='JOUD')

In [4]:
#(20)17(年) (第)25(号) といった感じ。
# 花王の、テレビ局ごとの指定している号。
kao_dict = dict(NTV=1725,
                TBS=1724,
                CXT=1727,
                EX=1724,
                TX=1722)

## ○平均値（号数）を算出する関数

In [5]:
import sys
sys.path.append('../..')
from util import dao

def add_other_gou(df, gou_dict, tv_dict, n_gou):
    housoukyoku = df.loc["放送局"].replace(' ','') # 空白をなくす。
    start_time = str(df["開始\r\n時間"]).replace(':', '')
    end_time = str(df["終了\r\n時間"]).replace(':', '')
    yobi = df["曜\r\n日"]
    
    year = '20'+str(gou_dict[housoukyoku])[:2]
    gou = gou_dict[housoukyoku]%100 # 下２桁を出すことで、号の情報を求める。
    
    housoubi = ''
    for i in range(n_gou):
        housoubi += detect_day(year, gou-(i+1), yobi) + ','
        
    query="SELECT AVG(setai) AS 世帯視聴率の平均 "+\
    "FROM vr_minute WHERE time >= "+ start_time +\
    " AND time <= " + end_time+\
    " AND media_id = '" +tv_dict[housoukyoku]+\
    "' AND housou_day IN (" + housoubi[:-1] + ")"
    
    df = dao.read_sql_data(query, db_type='db_prod_kanto')
    return round(df.iat[0,0], 1)

***
***

## ○実験

In [6]:
import time

In [7]:
import pandas as pd
df_kao = pd.read_excel('input/花王_201808.xlsx')
df_kao.head(3)

,ブランド 固定,視聴率 固定,JOB No. 提供No,区 分,放送局,放送日,曜 日,開始 時間,終了 時間,番組タイトル/提供名,...,タイム ランク,ブランド コード,ブランド名,PIB 係数,世帯,◆ＡＬＬ,女 18～34才,女 25～39才,女 35～49才,主婦
1,NaN,NaN,180801,S,NTV,2018/08/01,水,06:30,08:00,ＺＩＰ！,...,B,NaN,NaN,NaN,10.4,5.5,5.7,6.1,9.1,8.1
2,NaN,NaN,180801,S,NTV,2018/08/01,水,08:00,09:30,スッキリ（１部）,...,B,NaN,NaN,NaN,6.1,3.1,4.1,4.5,5.3,5.4
3,NaN,NaN,180801,S,NTV,2018/08/01,水,08:00,09:30,スッキリ（１部）,...,B,NaN,NaN,NaN,6.1,3.1,4.1,4.5,5.3,5.4


In [8]:
# 最初の3行で実験する。
df_sample = df_kao.head(3)

In [9]:
n_gou = 4 # 号数を4で指定する。

In [10]:
start = time.time() # 
df_sample['4_gou'] = df_sample.head().apply(lambda x:add_other_gou(x, kao_dict, tv_dict, n_gou), axis=1)
end = time.time()
print('処理時間:', end-start, '[s]')

処理時間: 21.028334617614746 [s]


/home/dataartist/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
df_sample

,ブランド 固定,視聴率 固定,JOB No. 提供No,区 分,放送局,放送日,曜 日,開始 時間,終了 時間,番組タイトル/提供名,...,ブランド コード,ブランド名,PIB 係数,世帯,◆ＡＬＬ,女 18～34才,女 25～39才,女 35～49才,主婦,4_gou
1,NaN,NaN,180801,S,NTV,2018/08/01,水,06:30,08:00,ＺＩＰ！,...,NaN,NaN,NaN,10.4,5.5,5.7,6.1,9.1,8.1,10.4
2,NaN,NaN,180801,S,NTV,2018/08/01,水,08:00,09:30,スッキリ（１部）,...,NaN,NaN,NaN,6.1,3.1,4.1,4.5,5.3,5.4,6.1
3,NaN,NaN,180801,S,NTV,2018/08/01,水,08:00,09:30,スッキリ（１部）,...,NaN,NaN,NaN,6.1,3.1,4.1,4.5,5.3,5.4,6.1


### 一致したので、正しそう！

***

### 【SQL文の書き方】
```
SELECT AVG(setai) AS '世帯視聴率の平均'
FROM vr_minute
WHERE time >= start_time AND time <= end_time
    AND media_id = housoukyoku
    AND housou_day IN housoubi
# GROUP BY housou_day 結局全部1分ずつの視聴率が出てるのなら必要ない。
# 入れるのなら、SELECT文の中にもhousou_dayが必要になる。
```